### note: change all of the paths to your correct path when loading in data

In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy as scp
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
import sklearn
from sklearn.linear_model import LinearRegression
import os
import Levenshtein

In [2]:
pd.options.display.max_rows = 600
pd.options.display.max_columns = 85

In [4]:
home_tmp = pd.read_csv('~/Dropbox/CDS-2019-AlbanyHub/Raw-Data/mismatched_home.csv', header = None)
cdbg_tmp = pd.read_csv('~/Dropbox/CDS-2019-AlbanyHub/Raw-Data/mismatched_cdbg.csv', header = None)
mismatched_home = home_tmp.iloc[:,0]
mismatched_cdbg = cdbg_tmp.iloc[:,0]

In [5]:
mismatched_home.shape[0] + mismatched_cdbg.shape[0]

99

In [7]:
df_utilities2012 = pd.read_csv('~/Dropbox/CDS-2019-AlbanyHub/Processed-Data/Totals2012.csv')
df_utilities2013 = pd.read_csv('~/Dropbox/CDS-2019-AlbanyHub/Processed-Data/Totals2013.csv')
df_utilities2014 = pd.read_csv('~/Dropbox/CDS-2019-AlbanyHub/Processed-Data/Totals2014.csv')
df_utilities2015 = pd.read_csv('~/Dropbox/CDS-2019-AlbanyHub/Processed-Data/Totals2015.csv')
df_utilities2016 = pd.read_csv('~/Dropbox/CDS-2019-AlbanyHub/Processed-Data/Totals2016.csv')
df_utilities2017 = pd.read_csv('~/Dropbox/CDS-2019-AlbanyHub/Processed-Data/Totals2017.csv')
df_utilities2018 = pd.read_csv('~/Dropbox/CDS-2019-AlbanyHub/Processed-Data/Totals2018.csv')
df_utilities2019 = pd.read_csv('~/Dropbox/CDS-2019-AlbanyHub/Processed-Data/Totals2019.csv')
df_utilities_list = [df_utilities2012,df_utilities2013,df_utilities2014,df_utilities2015,
                    df_utilities2016,df_utilities2017,df_utilities2018,df_utilities2019]

/anaconda3/envs/cds_env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/envs/cds_env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (4,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
list1 = []
list2 = []
for df_u in df_utilities_list:
    list1.append(df_u['Premise Address'])
for series_u in list1:
    for row in series_u:
        list2.append(row)
series_utilities = pd.Series(list2)

In [9]:
s_utilities = series_utilities.unique()

In [11]:
xlsx = pd.ExcelFile('~/Dropbox/CDS-2019-AlbanyHub/Raw-Data/All CED Funded Projects 2007-2017.xlsx')
df1_original = pd.read_excel(xlsx, 'HOME Funded Projects')
df2_original = pd.read_excel(xlsx, 'CDBG Funded Projects')
df_original = pd.concat([df1_original, df2_original])
df_original.index = range(850)

### helper functions

In [12]:
def lev_list(address_list, address, i):
    temp_list = []
    temp_list_u = []
    for u_address in address_list:
        if Levenshtein.distance(address, u_address.upper()) <= i:
            temp_list.append(address)
            temp_list_u.append(u_address.upper())
    temp_list_series = pd.Series(temp_list)
    temp_list_u_series = pd.Series(temp_list_u)
    return temp_list_u_series

In [13]:
def space_fix_utilities(s_util):
    s_util_to_series = []
    for address in s_util:
        full_address = address.split()
        length = len(full_address)
        word = full_address[0] + "  "
        for i in range(1, length-1):
            word += full_address[i] + " "
        word += " " + full_address[length-1]
        word = word.strip()
        s_util_to_series.append(word)
    return pd.Series(s_util_to_series)

In [14]:
def construct_fixed_addresses(list_series):
    tmp_to_series = []
    for (apt, dup, sno, snm, stp) in zip(list_series[0],list_series[1],list_series[2],
                                         list_series[3],list_series[4]):
        word = ""
        if sno != "NA":
            word += sno + "  "
        if snm != "NA":
            word += snm + "  "
        if stp != "NA":
            word += stp
        tmp_to_series.append(word.strip())
    return pd.Series(tmp_to_series)

In [15]:
def gen_list_none(address_list, mismatched, i):
    tmp_list = []
    for address in mismatched:
        if address == "UNKNOWN ADDRESS" or address == "SUPPRESSED ADDRESS":
            continue
        if lev_list(address_list, address, i).shape[0] == 0:
            tmp_list.append(address)
    return pd.Series(tmp_list)

### Read in more data

In [31]:
# s_utilities = space_fix_utilities(s_utilities)
df_parsehome = pd.read_csv('~/Dropbox/CDS-2019-AlbanyHub/Raw-Data/fixed_home_v02.csv')
df_parsecdbg = pd.read_csv('~/Dropbox/CDS-2019-AlbanyHub/Raw-Data/fixed_cdbg.csv')

In [18]:
s_parsehome = df_parsehome['Address']
s_parsecdbg = df_parsecdbg['Address']

In [19]:
def containsAPT(series):
    apt_list = []
    for address in series:
        if "APT" in address:
            apt_list.append(address)
    return pd.Series(apt_list)

In [20]:
def containsDUP(series):
    dup_list = []
    for address in series:
        full_address = address.split(' ')
        length = len(full_address)
        for i in range(length - 1):
            word = full_address[i]
            if len(word) == 1:
                if not ((word == 'N') or (word == 'E')
                or (word == 'S') or (word == 'W')):
                    dup_list.append(address)
    return pd.Series(dup_list)

In [21]:
def clean(series):
    tmp_list = []
    for address in series:
        full_address = address.split(' ')
        length = len(full_address)
        if any(char.isdigit() for char in full_address[length-1]):
            tmp_list.append(address)
        else:
            tmp_list.append("---")
    return pd.Series(tmp_list)

In [22]:
def splitter(series):
    apt_list = set(containsAPT(series))
    dup_list = set(containsDUP(series))
    sno_to_series = []
    snm_to_series = []
    stp_to_series = []
    apt_to_series = []
    dup_to_series = []

    for address in series:
        full_address = address.split()
        length = len(full_address)
        begin_index = 1
        end_index = length - 1
        #mutually exclusive apt and duplex
        if address == "UNKNOWN ADDRESS":
            sno_to_series.append("NA")
            snm_to_series.append("UNKNOWN ADDRESS")
            stp_to_series.append("NA")
            apt_to_series.append("NA")
            dup_to_series.append("NA")
            continue
        if address == "SUPPRESSED ADDRESS":
            sno_to_series.append("NA")
            snm_to_series.append("SUPPRESSED ADDRESS")
            stp_to_series.append("NA")
            apt_to_series.append("NA")
            dup_to_series.append("NA")
            continue
        if address in apt_list:
            apt_to_series.append(full_address[length-1])
            stp_to_series.append(full_address[length-3])
            end_index -= 2
        else:
            apt_to_series.append("NA")
            stp_to_series.append(full_address[length-1])
            
        if address in dup_list:
            dup_to_series.append(full_address[1])
            begin_index += 1
        else:
            dup_to_series.append("NA")
        sno_to_series.append(full_address[0])
        to_str = full_address[begin_index:end_index]
        word = ""
        for part in to_str:
            word += part + " "
        snm_to_series.append(word.strip())
    series_apt = pd.Series(apt_to_series)
    series_dup = pd.Series(dup_to_series)
    series_sno = pd.Series(sno_to_series)
    series_snm = pd.Series(snm_to_series)
    series_stp = pd.Series(stp_to_series)
    return (series_apt, series_dup, series_sno, series_snm, series_stp)

In [23]:
series_apt_home, series_dup_home, series_sno_home, series_snm_home, series_stp_home = splitter(s_parsehome)
series_apt_cdbg, series_dup_cdbg, series_sno_cdbg, series_snm_cdbg, series_stp_cdbg = splitter(s_parsecdbg)

In [24]:
home = [series_apt_home, series_dup_home, series_sno_home, series_snm_home, series_stp_home]
cdbg = [series_apt_cdbg, series_dup_cdbg, series_sno_cdbg, series_snm_cdbg, series_stp_cdbg]
for hom in home:
    print(hom.shape)
for cdb in cdbg:
    print(cdb.shape)

(286,)
(286,)
(286,)
(286,)
(286,)
(564,)
(564,)
(564,)
(564,)
(564,)


In [25]:
def checker(list_series):
    tmp_to_series = []
    for (apt, dup, sno, snm, stp) in zip(list_series[0],list_series[1],list_series[2],
                                         list_series[3],list_series[4]):
        word = ""
        if sno != "NA":
            word += sno + " "
        if dup != "NA":
            word += dup + " "
        word += snm + " "
        if stp != "NA":
            word += stp + " "
        if apt != "NA":
            word += "APT " + apt
        word = word.strip()
        tmp_to_series.append(word)
    return pd.Series(tmp_to_series)

In [26]:
s_parsehome[checker(home) != s_parsehome]

Series([], Name: Address, dtype: object)

In [27]:
df_all_addresses = pd.read_csv('~/Dropbox/CDS-2019-AlbanyHub/Processed-Data/all_addresses.csv')
df_all_addresses = df_all_addresses.drop('Unnamed: 0', axis = 1)
s_all_addresses = df_all_addresses['line1']

### Workflow

s_utilities is a list of all addresses that show up in the utilities data <br>
s_all_addresses is a list of all addresses in Albany <br>
df_original is a list of all housing projects <br>
lev_list takes in one of the two lists above, an address, and a threshold. <br>
If the address is within the threshold distance (string comparison) of an address in the two lists, it will print out.

### example (note the spacing difference between addresses in utilities and the full addr list)

In [28]:
lev_list(s_utilities, "2620  S MADISON  ST", 1)

0    2624  S MADISON  ST
1    2622  S MADISON  ST
2    2220  S MADISON  ST
3    2120  S MADISON  ST
4    2020  S MADISON  ST
dtype: object

In [29]:
lev_list(s_all_addresses, "2620 S MADISON ST", 1)

0    2624 S MADISON ST
1    2622 S MADISON ST
2    2620 S MADISON ST
3    2600 S MADISON ST
4    2220 S MADISON ST
5    2120 S MADISON ST
6    2020 S MADISON ST
dtype: object

So in the above example, 2620 S Madison St is found in the list of all addresses, but not found in the utilities data. Record this fact in the google doc. Then, look up the project type in the housing project data to also record what type of project it was. I'm looking to see if there is any trend with demolition or disposition projects not being found in utilities data.

### example (note the capitalization difference)

In [30]:
df_original[df_original['Address'].str.contains('Madison')]

,Parcel ID,Address,City,State,XY Coordinates,Plan Year,IDIS Project,Project,IDIS Activity ID,Activity Status,Program,Funded Amount,Draw Amount,Balance
57,00012/00004/011,2620 S. Madison,Albany,GA,"31.5319347, -84.1596916",2010,22,Acquisition/Rehabilitation,928,Completed,HOME,53553.46,53553.46,0.0
645,NaN,1308 S. Madison,Albany,GA,NaN,2014,9,Emergency Repairs,1247,Completed,CDBG,2987.00,2987.00,0.0
739,NaN,2221 S. Madison,Albany,GA,NaN,2015,16,Demolition,1334,Completed,CDBG,5092.00,5092.00,0.0


### Tests

In [32]:
mismatched_home

0             2620  S MADISON  ST
1               529  COCHRAN  AVE
2          2226  OLD DOMINION  DR
3             2701  DONCASTER  RD
4            250  BONNY VIEW  AVE
5            249  BONNY VIEW  AVE
6            248  BONNY VIEW  AVE
7            251  BONNY VIEW  AVE
8            253  BONNY VIEW  AVE
9            247  BONNY VIEW  AVE
10           254  BONNY VIEW  AVE
11           243  BONNY VIEW  AVE
12           245  BONNY VIEW  AVE
13           252  BONNY VIEW  AVE
14            2010  LAFAYETTE  CR
15              20  S JACKSON  ST
16               603  N HALEY  ST
17           1112  PEACHTREE  TER
18             2517  CARDINAL  DR
19            2006  LAFAYETTE  CR
20              818  WESTVIEW  DR
21              2501  REDWOOD  CR
22              933  RANDOLPH  DR
23            2201  CHAMPAGNE  DR
24              215  BROADWAY  ST
25              1003  HOLLAND  DR
26            2200  CHAMPAGNE  LN
27           1019  UNIVERSITY  DR
28            509  WESTOVER  BLVD
29            

In [148]:
lev_list(s_utilities, "2312  GLENEAGLE  DR", 1)

0    2312  GLENEAGLES  DR
dtype: object

In [150]:
lev_list(s_all_addresses, "2312 GLENEAGLE DR", 1)

0    2312 GLENEAGLES DR
dtype: object

In [151]:
df_original[df_original['Address'].str.contains('Gleneagle')]

,Parcel ID,Address,City,State,XY Coordinates,Plan Year,IDIS Project,Project,IDIS Activity ID,Activity Status,Program,Funded Amount,Draw Amount,Balance
232,00238/00047/003,2312 Gleneagle Drive Apt. B,Albany,GA,"31.5532888, -84.2080964",2016,19,Tenant Based Rental Assistance,1476,Completed,HOME,1026.31,1026.31,0.00
233,00238/00047/001,2316-B Gleneagles Drive,Albany,GA,"31.5532685, -84.2086317",2016,19,Tenant Based Rental Assistance,1412,Completed,HOME,7609.94,7609.94,0.00
270,00238/00047/003,2312 Gleneagles Drive Apt. B,Albany,GA,"31.5532888, -84.2080964",2017,11,Tenant Based Rental Assistance,1494,Open,HOME,6689.00,3866.53,2822.47
